<a href="https://colab.research.google.com/github/Jithin-Veeragandham/learn2code/blob/main/llama2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!CT_CUBLAS=1 pip install -q ctransformers --no-binary ctransformers &> /dev/null
!pip install -q flask pyngrok &> /dev/null

In [2]:
from ctransformers import AutoModelForCausalLM
model_id = "TheBloke/Llama-2-13B-chat-GGML"

# check ctransformers doc for more configs
config = {'max_new_tokens': 256, 'repetition_penalty': 1.1,
          'temperature': 0.1, 'stream': True}

llm = AutoModelForCausalLM.from_pretrained(
      model_id,
      model_type="llama",
      #lib='avx2', for cpu use
      gpu_layers=130, #110 for 7b, 130 for 13b
      **config
      )

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
from ctransformers import AutoModelForCausalLM
model_id = "TheBloke/CodeLlama-13B-Instruct-GGUF"
code_llm = AutoModelForCausalLM.from_pretrained(
      model_id,
      model_type="llama",
      #lib='avx2', for cpu use
      gpu_layers=130, #110 for 7b, 130 for 13b
      **config
      )

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
!pip install -q flask pyngrok
from flask import Flask, request, jsonify

app = Flask(__name__)

def generate_reply(prompt):
    tokens=llm.tokenize(prompt)
    output=""
    for token in llm.generate(tokens):
      output=output+llm.detokenize(token)
    return output

def generate_code(prompt):
    tokens=code_llm.tokenize(prompt)
    output=""
    for token in code_llm.generate(tokens):
      output=output+code_llm.detokenize(token)
    return output

@app.route('/generate_topics/', methods=['POST'])
def generate_topics():
    data = request.json
    language = data.get("language", "")
    difficulty = data.get("difficulty", "")
    num_topics = data.get("num_topics", "")
    prompt = f"List {num_topics} {difficulty} level {language} topics. Please provide names only, no descriptions."
    reply = generate_reply(prompt)
    return jsonify({"reply": reply})

@app.route('/generate_question/', methods=['POST'])
def generate_question():
    data = request.json
    language = data.get("language", "")
    concept = data.get("concept", "")

    prompt = f"give me a programming question utliziing {concept} in {language} without the solution, give me the question alone without any code"
    reply = generate_reply(prompt)
    return jsonify({"question": reply})

@app.route('/generate_code/', methods=['POST'])
def generate_question():
    data = request.json
    prompt = data.get("question", "")
    reply=generate_code(prompt)
    return jsonify({"code": reply})

import threading
def run():
    app.run(port=7000)
threading.Thread(target=run).start()


enter auth token here